# Procesamiento Avanzado y Modelado con BERT

#### **Objetivo :**
- Configurar el entorno para utilizar modelos avanzados como BERT.
- Preparar los datos procesados para ser ingresados al modelo.
- Entrenar un modelo de análisis de sentimientos basado en BERT.

#### **Celda 1: Configuración inicial**


In [ ]:
# Importar librerías necesarias
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Verificar si se dispone de GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Ruta al dataset procesado
data_path = r"../outputs/cleaned_data.csv"

# Cargar el dataset procesado
df = pd.read_csv(data_path)

# Inspeccionar las primeras filas del dataset
print("Primeras filas del dataset:")
print(df.head())
print("\nDistribución de sentimientos:")
print(df["sentiment"].value_counts())

#### **Celda 2: Preparar el tokenizer de BERT**


In [17]:
# Cargar el tokenizer preentrenado de BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Función para tokenizar los textos
def tokenize_texts(texts, tokenizer, max_len=128):
    input_ids, attention_masks = [], []
    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        input_ids.append(encoded["input_ids"])
        attention_masks.append(encoded["attention_mask"])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# Tokenizar los textos del dataset
print("Tokenizando los textos...")
input_ids, attention_masks = tokenize_texts(df["processed_text"].values, tokenizer)

# Mapear las etiquetas de sentimiento a valores numéricos
label_mapping = {"positivo": 0, "neutral": 1, "negativo": 2}
df["label"] = df["sentiment"].map(label_mapping)
labels = torch.tensor(df["label"].values)

print("Tokenización completa.")
print(f"Forma de input_ids: {input_ids.shape}")
print(f"Forma de attention_masks: {attention_masks.shape}")

Tokenizando los textos...
Tokenización completa.
Forma de input_ids: torch.Size([9252, 128])
Forma de attention_masks: torch.Size([9252, 128])


##### **Celda 3: Dividir los datos en entrenamiento y validación**


In [18]:
# Dividir los datos en entrenamiento y validación
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42
)

# Crear DataLoaders
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_data, sampler=SequentialSampler(val_data), batch_size=batch_size)

print("Datos divididos y DataLoaders creados.")

Datos divididos y DataLoaders creados.


#### **Celda 4: Configurar el modelo BERT**

In [19]:
# Cargar el modelo preentrenado con una capa de clasificación
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,  # Tres clases: positivo, neutral, negativo
    output_attentions=False,
    output_hidden_states=False,
)

# Mover el modelo al dispositivo seleccionado
model.to(device)
print("Modelo cargado y configurado.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo cargado y configurado.


#### **Celda 5: Configurar optimizador y bucle de entrenamiento**

In [ ]:
from transformers import AdamW

# Configurar optimizador
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

# Configurar scheduler para ajustar la tasa de aprendizaje
define_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Número de épocas
epochs = 3

# Bucle de entrenamiento
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        batch_input_ids, batch_masks, batch_labels = [item.to(device) for item in batch]

        # Reiniciar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_input_ids, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Pérdida promedio por época: {avg_loss:.4f}")


Epoch 1/3


c:\CondaEnvs\ia_env\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### **Celda 6: Evaluación del modelo**

In [ ]:
from sklearn.metrics import classification_report

# Evaluación en el conjunto de validación
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in val_dataloader:
        batch_input_ids, batch_masks, batch_labels = [item.to(device) for item in batch]

        outputs = model(batch_input_ids, attention_mask=batch_masks)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        labels = batch_labels.cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels)
        # Mostrar reporte de clasificación
print(classification_report(all_labels, all_preds, target_names=["positivo", "neutral", "negativo"]))

### **Notas finales:**
1. Este notebook entrena un modelo BERT para análisis de sentimientos con datos tokenizados y procesados previamente.
2. Configuramos tanto el entrenamiento como la evaluación con métricas estándar.
